# Декораторы


# Замыкания [Closures]
*In computer programming languages, a closure is a function together with a referencing environment of that function. A closure function is any function that uses a variable that is defined in an environment (or scope) that is external to that function, and is accessible within the function when invoked from a scope in which that free variable is not defined.*

Существования замыканий следует из правила LEGB, возможности оперировать с функциями как обьектами и того что области видимости в Питоне - статические.

In [ ]:
value = 1

def foo():
    
    print(value)
    
    def bar():
        print(value)
    
    bar()
    value = 2
    
foo()

UnboundLocalError: ignored

In [ ]:
multipliers = []

for m in range(10):
    multipliers.append(lambda x: x * m)

print(f'm = {m}')

print([multipliers[i](5) for i in range(5)])

m = 0

print([multipliers[i](5) for i in range(5)])


print(multipliers[3].__code__, multipliers[4].__code__)

m = 9
[45, 45, 45, 45, 45]
[0, 0, 0, 0, 0]
<code object <lambda> at 0x7f21df1ebdb0, file "<ipython-input-13-f8d6f518acee>", line 4> <code object <lambda> at 0x7f21df1ebdb0, file "<ipython-input-13-f8d6f518acee>", line 4>


In [ ]:
def function(a, b):
    return NAME

print('hello')

hello


In [ ]:
function(1, 2)

NameError: ignored

In [ ]:
NAME = 'Ilia'
function(1, 2)

'Ilia'

In [ ]:
def foo():
    x = 3
    def bar():
        print(x)
    x = 5
    return bar

bar_global = foo()
bar_global()

x = 9
bar_global()
print(f'x = {x}')

5
5
x = 9


In [ ]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

add_two = make_adder(2)

print(add_two(5))
print(add_two(7))

7
9


#### Функции могут замыкать одинаковые переменные

In [ ]:
def cell(value = 0):
    def Get():
        return value
    
    def Set(new_value):
        nonlocal value
        value = new_value
        return value
    
    return Get, Set

Get, Set = cell(10)
print(Get())

print(f'print(Get.__closure__[0].cell_contents) : {Get.__closure__[0].cell_contents}')

Set(20)
print(Get())

10
print(Get.__closure__[0].cell_contents) : 10
20


#### Посмотрим, что внутри замыкания

In [ ]:
print(Get.__closure__)
print(Get.__closure__[0].cell_contents)

(<cell at 0x7f21df1869d0: int object at 0x563a4b0abc60>,)
20


**\_\_closure\_\_** &mdash; список замкнутых переменных.<br>
Переменная представлена в виде класса **cell** с единственным полем **cell_contents**

In [ ]:
print(Get.__closure__ == Set.__closure__)
print(Get.__closure__[0] is Set.__closure__[0])

True
True


# Декораторы

Замыкания как способ быстро изменить поведение функции

In [ ]:
import sys  # посмотреть

def deprecate(func):
    def inner(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return inner

pprint = deprecate(print)

pprint([1, 2, 3])

[1, 2, 3]


print is deprecated


### Наблюдение

In [ ]:
import sys

def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper


@deprecated
def show(x):
    print(x)

show([1, 2, 3])

[1, 2, 3]


show is deprecated


### Проблема

In [ ]:
@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

wrapper
None


### Решение 1

In [ ]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


### Решение 2

In [ ]:
import functools

def deprecated(func):
    @functools.wraps(func) 
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


### Декораторы с аргументами

In [ ]:
def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('{} called with args {}, kwargs {}!'.format(func.__name__, args, kwargs), file = dest)
            return func(*args, **kwargs)
        return wrapper
    return wraps

@trace(sys.stdout) 
def f(x, test):
    if test > 1:
        return f(x, test / 2)

f('Hi!', test=42)

f called with args ('Hi!',), kwargs {'test': 42}!
f called with args ('Hi!', 21.0), kwargs {}!
f called with args ('Hi!', 10.5), kwargs {}!
f called with args ('Hi!', 5.25), kwargs {}!
f called with args ('Hi!', 2.625), kwargs {}!
f called with args ('Hi!', 1.3125), kwargs {}!
f called with args ('Hi!', 0.65625), kwargs {}!


#### Минизадача
Написать декоратор **once(function)**.<br>
Декоратор вызывает функцию только один раз.

In [ ]:
import functools 

def once(func): 
    def only_once(*args, **kwargs): 
        nonlocal called
        if not called: 
            called = True
            return func(*args, **kwargs) 
        
    called = False
    return only_once 

In [ ]:
@once
def foo():
    print('Hi!')

foo()
foo()
foo()
foo()

Hi!


In [ ]:
foo()

In [ ]:
func = decorator(func)

NameError: ignored

### Декораторам необязательно быть функциями

In [ ]:
from collections import Counter 

class Register(object):
    def __init__(self):
        self.stat = Counter()
        
    def __call__(self, func):
        nm = func.__name__
        def wrapper(*args, **kwrags):
            self.stat[nm] += 1
            return func(*args, **kwrags)
        return wrapper
    
    def __str__(self):
        result = 'fname\tcallcount\n'
        for name, count in self.stat.items():
            result += '{}:\t{}\n'.format(name, count)
        return result
    
register = Register()

In [ ]:

@register
def f(x):
    return x 

@register
def q(x):
    return q

f(1), q(2), q(4)
q(2), f(5)
print(register)

In [ ]:
#Обработка исключений

arr = [1,2,3]

try:
    arr[1] = 0
    #arr[3] = 0
    #tuple(arr)[1] = 0
    
except IndexError:
    print('except IndexError is executed if IndexError occurs in try')
    
except:
    print('except is executed if not listed error occurs')
    
else:
    print('else is executed if try worked')
    
finally:
    print('finally is always executed')

In [ ]:
print a